In [2]:
import os 
data_path = '/home/lfaivish/PycharmProjects/Deepshit/DATA_FOLDER'
os.chdir("C:\\Users\\awagner\\Documents\\DataScientists")

from spark import utils_function_spark
from spark import arrange_intervals

import gc
import numpy as np
from pyspark import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.sql.functions import ceil, unix_timestamp
from pyspark.sql.functions import rank
from pyspark.sql.functions import collect_list, array
from pyspark.mllib.linalg import Vectors, VectorUDT
from pyspark.sql.types import *
import time

from scipy.interpolate import interpolate
import pywt
from future.utils import lmap
import numpy as np
from functools import partial


In [3]:
spark = SparkSession.builder.appName("some_testing2").master("local").getOrCreate()
raw_data = spark.read.format('com.databricks.spark.csv').option("header", "True").option("delimiter", ",")\
                      .load('C:/Users/awagner/Desktop/For_Tom/'+'AllLabData.csv')


In [4]:

t = time.time()
raw_data = arrange_intervals.casting_data(raw_data)
print('stage1 ' + str(time.time() - t))
data_frame = arrange_intervals.make_inteval_per_task(raw_data)
print('stage2 ' + str(time.time() - t))

data_frame = arrange_intervals.tasks_to_intervals(data_frame,
                                                  slide = 2.5, window = 5)
print('tasks_to_intervals ' + str(time.time() - t))

data_frame = arrange_intervals.get_features(data_frame)
print('stage4 ' + str(time.time() - t))




stage1 0.799109935760498
stage2 334.1456081867218
stage3 346.24051666259766
stage4 346.83390712738037


In [5]:
del raw_data
gc.collect()

data_frame.show(5)
print('data_frame_show ' + str(time.time() - t))

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 key|    rel_features_ver|   cont_features_ver|    rel_features_hor|   cont_features_hor|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[3.1923658E7,0.0,...|[0.0,0.0060506603...|[8.326673E-17,0.0...|[0.0,0.07433381,0...|[2.7755576E-17,0....|
|[3.1923658E7,0.0,...|[0.0,0.024748635,...|[5.551115E-17,0.2...|[0.0,0.038538158,...|[1.6653345E-16,0....|
|[3.1923658E7,0.0,...|[0.0,0.03884047,0...|[5.551115E-17,0.4...|[0.0,0.034145087,...|[6.938894E-17,0.1...|
|[3.1923658E7,0.0,...|[0.0,0.056474645,...|[3.8857806E-16,0....|[0.0,0.024905436,...|[6.938894E-17,0.1...|
|[3.1923658E7,0.0,...|[0.0,0.015611432,...|[4.1633363E-17,0....|[0.0,0.05458911,0...|[5.551115E-17,0.3...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

data_frame_s